In [1]:
import astroprov
from astropy.io import fits
from astropy.io.ascii import SExtractor
from astropy.stats import sigma_clipped_stats
from astropy.coordinates import SkyCoord
import astropy.coordinates as coord

import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
import os
from photutils import DAOStarFinder
from photutils import CircularAperture
from photutils import aperture_photometry
#import subproc

import numpy as np
from astropy import wcs
from astropy.io import fits
import sys

from astroquery.simbad import Simbad

import matplotlib.pyplot as plt
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from astropy.table import Table, vstack
from astropy.table import Column

import collections
import subprocess
from astropy.io import ascii
import time
from astropy.utils.data import clear_download_cache
from astroquery.gaia import Gaia

Created TAP+ (v1.0.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: False
	Port: 80
	SSL Port: 443


In [2]:
clear_download_cache()

In [3]:
def alterDefault(defaultDir,valList,attributeList,catName):
    
    with open(defaultDir+'/default.sex','r') as f:
        data  = f.read()
    for x in range(len(valList)):
        nameLoc = data.find(attributeList[x]) + len(attributeList[x])
        endLoc = data[nameLoc:].find('#') + nameLoc
        newData = data[:nameLoc] + ' '+str(valList[x])+' ' + data[endLoc:]
        data = newData
    
    with open(defaultDir+'/default.sex','r') as f:
        endData = f.read()
    cname = 'CATALOG_NAME'
    nameLoc = endData.find(cname) + len(cname)
    endLoc = data[nameLoc:].find('#') + nameLoc
    newData = data[:nameLoc] + ' ' +catName+ ' ' + data[endLoc:]
    
    with open(defaultDir+'/default.sex','r') as f:
        endData = f.read()
    cname = 'PARAMETERS_NAME'
    nameLoc = endData.find(cname) + len(cname)
    endLoc = data[nameLoc:].find('#') + nameLoc
    newData = data[:nameLoc] + ' autodefault.param ' + data[endLoc:]
    
    #print(newData)
    with open(defaultDir+'/default.sex','w') as f:
        f.write(newData)

In [4]:
def sextractor(sextractoryDir,imagename,cataloguename,imageDir='/home/mj1e16/keplerImages/'):
    
    subprocess.call(['sex',imageDir+imagename])
    assoc = Table.read(cataloguename,format='ascii.sextractor')

#     os.chdir(imageDir)
#     hdu_list = fits.open(imagename) # open image header
#     imagedata = (hdu_list[0].data) 
    
#     fig, ax = plt.subplots(figsize=(10, 10))
#     positions = (assoc['X_IMAGE'], assoc['Y_IMAGE'])
#     apertures = CircularAperture(positions, r=4.)
#     norm = ImageNormalize(stretch=SqrtStretch())
#     ax.imshow(imagedata, cmap='Greys', origin='lower', vmin=-100,vmax=100)#norm=norm)
#     apertures.plot(color='blue', lw=1.5, alpha=0.5)
#     #plt.savefig('fits.png',dpi=100)
#     plt.show()
    
#     print(len(assoc))
#     mags = assoc['MAG_APER']
#     plt.hist(mags,bins='auto')
#     plt.show()
    return assoc

In [5]:
def makeDS9RegFile(sexTabList,fileNameBase,tabType):
    for tables in range(len(sexTabList)):
        if tabType == 'dao':
            xcoords = sexTabList[tables]['xcentroid']
            ycoords = sexTabList[tables]['ycentroid']
        elif tabType == 'sex':
            xcoords = sexTabList[tables]['X_IMAGE']
            ycoords = sexTabList[tables]['Y_IMAGE']
        elif tabType == 'iraf':
            xcoords = sexTabList[tables]['X_POS']
            ycoords = sexTabList[tables]['Y_POS']        
        else:
            print('tab type error')
            break
        bigString = 'global color=lightgreen\nimage\n'
        for x in range(len(xcoords)):
            bigString += 'circle({},{},5)\n'.format(xcoords[x],ycoords[x])
        fileName = fileNameBase + '.reg'
        with open(fileName,'w') as f:
            f.write(bigString)
#         print(fileName)
#         print(os.listdir('.'))

In [6]:
def load_wcs_from_file(filename,pixCoord):
    # Load the FITS hdulist using astropy.io.fits
    hdulist = fits.open(filename)

    # Parse the WCS keywords in the primary HDU
    w = wcs.WCS(hdulist[0].header)

    # Print out the "name" of the WCS, as defined in the FITS header
    #print(w.wcs.name)

    # Print out all of the settings that were parsed from the header
    w.wcs.print_contents()

    # Three pixel coordinates of interest.
    # Note we've silently assumed a NAXIS=2 image here.
    # Note also that the pixel coordinates are pairs of [X, Y], 
    # and since WCS built from FITS header automatically has
    # origin set to 1, [0, 0] is actually not inside the image.
    pixcrd = np.array(pixCoord, dtype=np.float_)

    # Convert pixel coordinates to world coordinates
    # The second argument is "origin" -- in this case we're declaring we
    # have 1-based (Fortran-like) coordinates.
    world = w.wcs_pix2world(pixcrd, 1)
    #print(world)
    return(world)

In [7]:
def queryObjectRegion(outputTable,imageName,inCat,searchRadius=5*u.arcsecond,imDir='/home/mj1e16/simages/'):
    imageCoords = []
    r = searchRadius
    
    if inCat == 'dao':
        xlabel = 'xcentroid'
        ylabel = 'ycentroid'
        label = 'id'
        
    if inCat == 'sex':
        xlabel = 'X_IMAGE'
        ylabel = 'Y_IMAGE'
        label = 'NUMBER'
    flux = []   
    tableLength = len(outputTable)
    xpos = outputTable[xlabel]
    ypos = outputTable[ylabel]
    for x in range(tableLength):
        xpos = outputTable[xlabel][x]
        ypos = outputTable[ylabel][x]
        flux.append(outputTable['FLUX_MAX'][x])
        if 20 < ypos < 1040:
            if 12 < xpos < 1110:
                imageCoords.append([xpos,ypos])
                # this may need to change per image but I hope not

    coords = load_wcs_from_file(imDir+imageName,imageCoords)
    customSimbad = Simbad()
    customSimbad.add_votable_fields('ra(d)','dec(d)','otype')
    customSimbad.remove_votable_fields('coordinates')
            
    result_table = Table([['-'],[0],[0],['-'],[0],[0],[0],['-']],names=('MAIN_ID','RA_d','DEC_d','OTYPE','xcentroid','ycentroid','flux','IMAGE'),dtype=('object','float64','float64','object','float64','float64','float64','object'))

    # change to update table and query in one go for the speed
#     for x in range(len(imageCoords)):
#         try:
#             c = SkyCoord(coords[x][0],coords[x][1],frame='icrs',unit='deg')
#             rt = customSimbad.query_region(c,radius=r)
#             rowlist = []
#             for y in range(len(rt[0])):
#                 rowlist.append(rt[0][y])
#             rowlist.append(imageCoords[x][0])
#             rowlist.append(imageCoords[x][1])
#             rowlist.append(flux[x])
#             rowlist.append(imageName)
#             result_table.add_row(rowlist)

#         except:
#             result_table.add_row(['N/A',[0],[0],'Object Not Found',[imageCoords[x][0]],[imageCoords[x][1]],flux[x],imageName])

   #result_table = Simbad.query_region(coord.SkyCoord
    for x in range(len(imageCoords)):
        try:
            c = SkyCoord(coords[x][0],coords[x][1],frame='icrs',unit='deg')
            rt = customSimbad.query_region(c,radius=r)
            rowlist = []
            for y in range(len(rt[0])):
                rowlist.append(rt[0][y])
            rowlist.append(imageCoords[x][0])
            rowlist.append(imageCoords[x][1])
            rowlist.append(flux[x])
            rowlist.append(imageName)
            result_table.add_row(rowlist)

        except:
            #print('searchfailed')
            result_table.add_row(['N/A',[0],[0],'Object Not Found',[imageCoords[x][0]],[imageCoords[x][1]],flux[x],imageName])

#     hdu_list = fits.open(imageName)
#     imagedata = (hdu_list[0].data) 
#     fig, ax = plt.subplots(figsize=(10, 10))
#     positions = (outputTable[xlabel], outputTable[ylabel])
#     apertures = CircularAperture(positions, r=4.)
#     norm = ImageNormalize(stretch=SqrtStretch())
#     ax.imshow(imagedata, cmap='Greys', origin='lower', vmin=-100,vmax=100)#norm=norm)
#     apertures.plot(color='blue', lw=1.5, alpha=0.5)
#     #plt.savefig('fits.png',dpi=100)
#     plt.show()
    
    return result_table

In [8]:
def runAndShow(image,cat,settings=[10,5,64,3],imDir='/home/mj1e16/Simages/',sexDir='/home/mj1e16/sextractor/sextractor-master/config/'):
    alterDefault(sexDir,settings,['DETECT_THRESH','DETECT_MINAREA','BACK_SIZE','BACK_FILTERSIZE'],catName)
    tab = sextractor(sexDir,image,cat,imageDir=imDir)
    #print(tab)
    pan = tab.to_pandas()
    pan = pan[pan['MAG_BEST'] != 99.0]
    pan = pan[pan['ELLIPTICITY'] < np.median(pan['ELLIPTICITY'])]
    pan = pan[pan['ELONGATION'] < np.median(pan['ELONGATION'])]

#     pan = pan[pan['X_IMAGE'] < 610.]
#     pan = pan[pan['X_IMAGE'] > 570.]
#     pan = pan[pan['Y_IMAGE'] < 300.]
#     pan = pan[pan['Y_IMAGE'] > 250.]
    tab = Table.from_pandas(pan)
    print(len(tab))
    try:
        searchTab = queryObjectRegion(tab,image,'sex',imDir=imDir)
        return searchTab
    except:
        print('search failed')
        return('failed')
        pass
#     makeDS9RegFile([tab],'/home/mj1e16/simages/autosextest','sex')
#     subprocess.call(['ds9',imDir+image,'-regions','/home/mj1e16/simages/autosextest.reg'])
    

In [9]:
def smollBoi(image,imDir,cat='testauto.cat',sexDir='/home/mj1e16/sextractor/sextractor-master/config/'):
    tab = sextractor(sexDir,image,cat,imageDir=imDir)
    if len(tab) != 0:
        pan = tab.to_pandas()
        pan = pan[pan['MAG_BEST'] != 99.0]
        pan = pan[pan['ELLIPTICITY'] < np.median(pan['ELLIPTICITY'])]
        pan = pan[pan['ELONGATION'] < np.median(pan['ELONGATION'])]
        tab = Table.from_pandas(pan)
        xpos = tab['X_IMAGE']
        ypos = tab['Y_IMAGE']
        coords = [[x,y] for x,y in zip(xpos,ypos)]
        imageCoords = load_wcs_from_file(imDir+image,coords)
        ra = [x[0] for x in imageCoords]
        dec = [x[1] for x in imageCoords]
        tab['RA'] = ra
        tab['DEC'] = dec
        name = findName(image)
        namelist = [name]*len(ra)
        tab['NAME'] = namelist
        #print(ra,dec)
    return tab

In [10]:
def findName(image):
    hdu = fits.open('/data/mj1e16/kepler/properDiff/'+image)
    head = hdu[0].header['DIFFCMD']
    locator1 = '/data/mj1e16/kepler/cal/'
    locator2 = ']'
    loc1 = head.find(locator1) + len(locator1)
    loc2 = head.find(locator2) + 1
    return head[loc1:loc2]

In [11]:
sexDir = '/home/mj1e16/sextractor/sextractor-master/config/'
catName = 'testauto.cat'
imdir = '/home/mj1e16/Simages/'

In [12]:
diffDir = '/data/mj1e16/kepler/properDiff/'
dirlist = os.listdir(diffDir)
print(len(dirlist))
dirlist = [x for x in dirlist if '.fits' in x]
print(len(dirlist))


4453
4452


In [13]:
tabTot =  ascii.read('/home/mj1e16/outTablesImproved/final/magone.csv')

In [13]:
tabTot = ascii.read('/home/mj1e16/outTablesImproved/final/100sn.csv')

In [14]:
RA = tabTot['RA']
DEC = tabTot['DEC']
width = u.Quantity(5., u.arcsecond)
height = u.Quantity(5., u.arcsecond)
#coord = [str(RA[0])+','+str(DEC[0])]
for x in range(len(RA)):
    coord = SkyCoord(ra=RA[x], dec=DEC[x], unit=(u.degree, u.degree), frame='icrs')
    if x ==0:
        result_table = Gaia.query_object_async(coordinate=coord, width=width, height=height)
    else:
        newtab = Gaia.query_object_async(coordinate=coord, width=width, height=height)
        if len(newtab) != 0:
            result_table.add_row(newtab[0])

Query finished.
Query finished.
Query finished.
Query finished.
Query finished.
Query finished.
Query finished.
Query finished.
Query finished.
Query finished.
Query finished.
Query finished.
Query finished.
Query finished.
Query finished.
Query finished.
Query finished.
Query finished.
Query finished.
Query finished.
Query finished.
Query finished.
Query finished.
Query finished.
Query finished.
Query finished.
Query finished.
Query finished.


In [14]:
print(len(tabTot))

1459


In [ ]:
# from astroquery.gaia import Gaia
# Gaia.login()

# table = file or astropy.table
# Gaia.upload_table(upload_resource=table, table_name='my_sources')

# # the table will be uploaded into the user private space into the database
# # the table can be referenced as <database user schema>.<table_name>

# full_qualified_table_name = 'user_<your_login_name>.my_sources'
# xmatch_table_name = 'xmatch_table'
# Gaia.cross_match(full_qualified_table_name_a=full_qualified_table_name, \
#               full_qualified_table_name_b='gaiadr2.gaia_source', \
#               results_table_name=xmatch_table_name, radius=1.0)

# # Once you have your cross match finished, you can obtain the results:
# xmatch_table = 'user_<your_login_name>.' + xmatch_table_name
# query = 'SELECT c."dist", a.*, b.* FROM gaiadr2.data_source AS a, '+\
# full_qualified_table_name+' AS b, '+\
# xmatch_table+' AS c '+\
# 'WHERE (c.gaia_source_source_id = a.source_id AND c.my_sources_my_sources_oid = b.my_sources_oid)'
# job = Gaia.launch_job(query=query)
# results = job.get_resultsjob)
# print(results)

In [ ]:
coord = SkyCoord(ra=280, dec=-60, unit=(u.degree, u.degree), frame='icrs')
width = u.Quantity(0.1, u.deg)
height = u.Quantity(0.1, u.deg)
r = Gaia.query_object_async(coordinate=coord, width=width, height=height)
r.pprint()

In [18]:
redos = []
segs = 1000
for x in range(segs):
    start = int(x*(bigLength/segs))
    end = int((x+1)*(bigLength/segs))
    smollTab = tabTot[start:end]
    RA = smollTab['RA']
    DEC = smollTab['DEC']
    if x == 0:
        result_table = Simbad.query_region(coord.SkyCoord(ra=RA,dec=DEC,unit=(u.deg,u.deg),frame='fk5'),radius=5*u.arcsec)
    else:
        #try:
        newtab = Simbad.query_region(coord.SkyCoord(ra=RA,dec=DEC,unit=(u.deg,u.deg),frame='fk5'),radius=5*u.arcsec)
        result_table = vstack([result_table,newtab])
        print('Sucessful',x)
#         except ConnectionError:
#             print('Failed',x)
#             redos.append(x)
print('Sucessful: {}'.format(segs-x))
print('Failed: {}'.format(x))

/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/astroquery/simbad/core.py:136: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): '19:15:13.9971 +41:34:20.0765': No astronomical object found :  
  (error.line, error.msg))
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/astroquery/simbad/core.py:136: UserWarning: Warning: The script line number 8 raised an error (recorded in the `errors` attribute of the result table): '19:21:43.2615 +41:05:41.2633': No astronomical object found :  
  (error.line, error.msg))
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/astroquery/simbad/core.py:136: UserWarning: Warning: The script line number 11 raised an error (recorded in the `errors` attribute of the result table): '19:21:17.0396 +41:03:27.2523': No astronomical object found :  
  (error.line, error.msg))
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-pack

('Sucessful', 1)

/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/astroquery/simbad/core.py:136: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): '18:42:46.2101 +44:04:52.6381': No astronomical object found :  
  (error.line, error.msg))
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/astroquery/simbad/core.py:136: UserWarning: Warning: The script line number 5 raised an error (recorded in the `errors` attribute of the result table): '18:44:53.4454 +43:34:51.3244': No astronomical object found :  
  (error.line, error.msg))
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/astroquery/simbad/core.py:136: UserWarning: Warning: The script line number 9 raised an error (recorded in the `errors` attribute of the result table): '18:44:47.7093 +43:35:09.1375': No astronomical object found :  
  (error.line, error.msg))
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packa


('Sucessful', 2)


/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/astroquery/simbad/core.py:136: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): '18:47:10.6785 +47:09:55.6909': No astronomical object found :  
  (error.line, error.msg))
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/astroquery/simbad/core.py:136: UserWarning: Warning: The script line number 5 raised an error (recorded in the `errors` attribute of the result table): '18:48:29.2053 +47:18:31.3264': No astronomical object found :  
  (error.line, error.msg))
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/astroquery/simbad/core.py:136: UserWarning: Warning: The script line number 8 raised an error (recorded in the `errors` attribute of the result table): '18:44:40.1278 +46:47:26.4974': No astronomical object found :  
  (error.line, error.msg))
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packa

...

/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/astroquery/simbad/core.py:136: UserWarning: Warning: The script line number 5 raised an error (recorded in the `errors` attribute of the result table): '19:28:14.7934 +46:19:48.9184': No astronomical object found :  
  (error.line, error.msg))
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/astroquery/simbad/core.py:136: UserWarning: Warning: The script line number 8 raised an error (recorded in the `errors` attribute of the result table): '19:29:15.6158 +46:01:52.5111': No astronomical object found :  
  (error.line, error.msg))
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/astroquery/simbad/core.py:136: UserWarning: Warning: The script line number 10 raised an error (recorded in the `errors` attribute of the result table): '19:29:25.1181 +45:57:17.9373': No astronomical object found :  
  (error.line, error.msg))
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-pack

('Sucessful', 6)


('Sucessful', 7)


/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/astroquery/simbad/core.py:136: UserWarning: Warning: The script line number 4 raised an error (recorded in the `errors` attribute of the result table): '19:38:29.1156 +46:14:10.2187': No astronomical object found :  
  (error.line, error.msg))
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/astroquery/simbad/core.py:136: UserWarning: Warning: The script line number 9 raised an error (recorded in the `errors` attribute of the result table): '19:36:50.5062 +46:47:59.8728': No astronomical object found :  
  (error.line, error.msg))
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/astroquery/simbad/core.py:136: UserWarning: Warning: The script line number 12 raised an error (recorded in the `errors` attribute of the result table): '19:36:10.2231 +47:02:40.4141': No astronomical object found :  
  (error.line, error.msg))
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-pack

MemoryError: 

In [1]:
a = 0

In [ ]:
Simbad.ROW_LIMIT = 1

imageCoords = []
inCat = 'sex'
imageName = 'diff_44_1.fits'
if inCat == 'dao':
    xlabel = 'xcentroid'
    ylabel = 'ycentroid'
    label = 'id'

if inCat == 'sex':
    xlabel = 'X_IMAGE'
    ylabel = 'Y_IMAGE'
    label = 'NUMBER'
flux = []   
tableLength = len(outTab)
xpos = totTab[xlabel]
ypos = totTab[ylabel]
flux = totTab['FLUX']

imageCoords = [[xp,yp] for xp,yp in zip(xpos,ypos)]
coords = load_wcs_from_file(imdir+imageName,imageCoords)
print(len(coords))
RA = [x[0] for x in coords]
DEC= [x[1] for x in coords]

result_table = Simbad.query_region(coord.SkyCoord(ra=RA,dec=DEC,unit=(u.deg,u.deg),frame='fk5'),radius=5*u.arcsec)

In [18]:
ascii.write(totTab,'sexdiffOut.csv',format='csv',fast_writer=False)

In [8]:
# t = time.time()

# for x in range(len(dirlist)):
#     if x == 0:
#         totTab = runAndShow(dirlist[x],catName,imDir=diffDir)
#     else:
#         outTab = runAndShow(dirlist[x],catName,imDir=diffDir)
#         if outTab != 'failed':
#             totTab = vstack([totTab,outTab])
#     if x%10 ==0:
#         percentage = x/4453.
#         print('{}% complete'.format(percentage))
#         print('runtime = {}'.format(time.time()-t))
        
# print(time.time() -t)

In [22]:
import sys
sys.getsizeof(outTab)

64

In [58]:
outTab = runAndShow('diff_44_1.fits',catName)

95


In [23]:
outTab

MAIN_ID,RA_d,DEC_d,OTYPE,xcentroid,ycentroid,flux,IMAGE
object,float64,float64,object,float64,float64,float64,object
-,0.0,0.0,-,0.0,0.0,0.0,-
N/A,0.0,0.0,Object Not Found,723.6713,27.216,366.942,diff_imNo42_channel32_ccd56.fits
N/A,0.0,0.0,Object Not Found,906.6633,22.0116,708.1851,diff_imNo42_channel32_ccd56.fits
N/A,0.0,0.0,Object Not Found,282.0037,23.9917,252.1652,diff_imNo42_channel32_ccd56.fits
N/A,0.0,0.0,Object Not Found,730.4838,33.1302,472.4556,diff_imNo42_channel32_ccd56.fits
N/A,0.0,0.0,Object Not Found,916.8593,25.8058,262.0544,diff_imNo42_channel32_ccd56.fits
N/A,0.0,0.0,Object Not Found,1037.9966,37.5477,184.2295,diff_imNo42_channel32_ccd56.fits
N/A,0.0,0.0,Object Not Found,277.9948,31.9981,513.701,diff_imNo42_channel32_ccd56.fits
N/A,0.0,0.0,Object Not Found,1037.019,33.3124,429.2621,diff_imNo42_channel32_ccd56.fits


In [15]:
Simbad.ROW_LIMIT = 1

imageCoords = []
inCat = 'dao'
imageName = 'diff_44_1.fits'
if inCat == 'dao':
    xlabel = 'xcentroid'
    ylabel = 'ycentroid'
    label = 'id'

if inCat == 'sex':
    xlabel = 'X_IMAGE'
    ylabel = 'Y_IMAGE'
    label = 'NUMBER'
flux = []   
tableLength = len(outTab)
xpos = outTab[xlabel]
ypos = outTab[ylabel]
flux = outTab['flux']

imageCoords = [[xp,yp] for xp,yp in zip(xpos,ypos)]
coords = load_wcs_from_file(imdir+imageName,imageCoords)
print(len(coords))
RA = [x[0] for x in coords]
DEC= [x[1] for x in coords]

result_table = Simbad.query_region(coord.SkyCoord(ra=RA,dec=DEC,unit=(u.deg,u.deg),frame='fk5'),radius=5*u.arcsec)


77


/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/astroquery/simbad/core.py:136: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): '19:31:59.7575 +44:05:50.7695': No astronomical object found :  
  (error.line, error.msg))
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/astroquery/simbad/core.py:136: UserWarning: Warning: The script line number 5 raised an error (recorded in the `errors` attribute of the result table): '19:29:27.3369 +44:44:39.8624': No astronomical object found :  
  (error.line, error.msg))
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/astroquery/simbad/core.py:136: UserWarning: Warning: The script line number 7 raised an error (recorded in the `errors` attribute of the result table): '19:23:46.5243 +44:20:58.4068': No astronomical object found :  
  (error.line, error.msg))
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packa

In [49]:
c = SkyCoord(ra= )

95

In [16]:
len(result_table)

1

In [17]:
result_table

MAIN_ID,RA,DEC,RA_PREC,DEC_PREC,COO_ERR_MAJA,COO_ERR_MINA,COO_ERR_ANGLE,COO_QUAL,COO_WAVELENGTH,COO_BIBCODE
,"""h:m:s""","""d:m:s""",,,mas,mas,deg,,,
object,unicode13,unicode13,int16,int16,float32,float32,int16,unicode1,unicode1,object
2MASS J19275592+4421299,19 27 55.9298,+44 21 30.025,14,14,0.019,0.021,90,A,O,2018yCat.1345....0G


In [12]:
a = [0,1,2]
b = [10,1,2]
x = 0
c = lambda a, b: [a[x],b[x]]
print(c(a,b))

[0, 10]


In [13]:
c = [[val,pair] for val, pair in zip(a,b)]
print(c)

[[0, 10], [1, 1], [2, 2]]


In [10]:
a = lambdafunc(a,b,x)
print(a)

<function <lambda> at 0x7fda24594848>


In [1]:
dirlist = os.listdir(imdir)
dirlist = [x for x in dirlist if x[-4:] =='fits'] # check
#print(dirlist)


for im in range(len(dirlist)):
    if im == 0:
        outTab = runAndShow(dirlist[im],catName,imDir=imdir)
    else:
        newTab = runAndShow(dirlist[im],catName,imDir=imdir)
        try:
            outTab = vstack([outTab,newTab])
        except:
            print('No transients found in Image '+dirlist[im])
df = outTab.to_pandas()
df = df[df['OTYPE'] == 'Object Not Found']
dfsort = df.sort_values(['FLUX_MAX'])

In [69]:
outPan = outTab.to_pandas()
sortTab = outPan.sort_values(['ELONGATION'])


In [29]:
dirlist = os.listdir(imdir)
dirlist = [x for x in dirlist if x[-4:] =='fits'] # check
print(dirlist)

['difkplr2009115080620_ffi-cal57.fits', 'difkplr2009115053616_ffi-cal68.fits', 'difkplr2010140101631_ffi-cal64.fits', 'difkplr2012004204112_ffi-cal15.fits', 'difkplr2012179140901_ffi-cal84.fits', 'difkplr2011177110110_ffi-cal80.fits', 'difkplr2010326181728_ffi-cal50.fits', 'difkplr2010265195356_ffi-cal76.fits', 'difkplr2009322233047_ffi-cal10.fits', 'difkplr2012151105138_ffi-cal45.fits', 'difkplr2012277203051_ffi-cal76.fits', 'difkplr2010078174524_ffi-cal84.fits', 'difkplr2009115173611_ffi-cal24.fits', 'difkplr2010111125026_ffi-cal56.fits', 'difkplr2012179140901_ffi-cal20.fits', 'difkplr2010174164113_ffi-cal30.fits', 'difkplr2009114204835_ffi-cal3.fits', 'difkplr2009292020429_ffi-cal71.fits', 'difkplr2010203012215_ffi-cal34.fits', 'difkplr2012277203051_ffi-cal34.fits', 'difkplr2009170043915_ffi-cal66.fits', 'difkplr2012277203051_ffi-cal16.fits', 'difkplr2011024134926_ffi-cal79.fits', 'difkplr2010203012215_ffi-cal25.fits', 'difkplr2009260000800_ffi-cal45.fits', 'difkplr2010326181728_ffi

In [70]:
sortTab

,NUMBER,X_IMAGE,Y_IMAGE,ALPHA_J2000,DELTA_J2000,FLUX_APER,FLUXERR_APER,MAG_APER,FLUX_RADIUS,FWHM_IMAGE,BACKGROUND,THRESHOLD,FLUX_MAX,ISOAREA_IMAGE,A_IMAGE,B_IMAGE,THETA_IMAGE,FLAGS,ELONGATION,ELLIPTICITY
1626,2889,407.4532,409.4460,0.0,0.0,1227.86600,9.083923,-7.7229,0.804,0.98,-0.669483,6.13679,366.29710,8,0.870,0.869,-70.58,0,1.001,0.001
2449,4183,480.9913,582.9999,0.0,0.0,147.95830,9.065479,-5.4253,0.419,0.00,-0.420949,6.13679,129.85340,3,0.715,0.714,71.92,0,1.002,0.002
3449,5657,636.6011,667.0096,0.0,0.0,927.34060,9.065479,-7.4181,0.722,1.07,-0.422779,6.13679,390.62750,9,0.852,0.850,48.01,0,1.002,0.002
2948,4872,177.2634,779.3145,0.0,0.0,4380.21600,9.037742,-9.1037,0.820,1.69,-0.285206,6.13679,1184.95400,10,0.921,0.919,65.99,0,1.002,0.002
169,296,952.9039,55.9717,0.0,0.0,11128.91000,9.046997,-10.1161,0.956,2.55,-0.037754,6.13679,2427.59500,12,0.973,0.972,-56.56,0,1.002,0.002
3336,5470,415.5953,684.6047,0.0,0.0,731.91220,9.065478,-7.1611,0.788,0.98,-0.372609,6.13679,252.34390,7,0.866,0.865,56.00,0,1.002,0.002
808,1527,277.9213,219.6775,0.0,0.0,2729.95500,9.065479,-8.5904,0.737,1.34,-0.762254,6.13679,1094.57900,8,0.904,0.902,62.21,3,1.002,0.002
1830,3211,141.9317,445.3230,0.0,0.0,2254.89200,9.037742,-8.3828,1.219,2.12,-0.613897,6.13679,785.11540,13,1.150,1.148,71.16,0,1.002,0.002
3319,5445,364.4733,689.9902,0.0,0.0,669.48400,9.074706,-7.0644,0.734,1.19,-0.445358,6.13679,239.15020,7,0.854,0.852,-37.33,0,1.002,0.002
2299,3959,824.2881,548.0385,0.0,0.0,807.67320,9.065479,-7.2681,0.708,1.15,-0.340390,6.13679,358.38990,8,0.851,0.848,-65.54,0,1.003,0.003


In [99]:
print(np.median(outTab['ELLIPTICITY']))

nan


/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/numpy/core/fromnumeric.py:2954: RuntimeWarning: Mean of empty slice.
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [100]:
print(np.median(outTab['ELONGATION']))
print(np.mean(outTab['ELONGATION']))

nan
nan
